In [1]:
import robotic as ry
from utility import *
from bottleneck_analysis import bottleneck_vertices_from_config

In [2]:
PUZZLE_NAME = "p6-wall"

In [3]:
config = ry.Config()
config.addFile(f"puzzles/{PUZZLE_NAME}.g")

EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"

GAP = config.getFrame(EGO_NAME).getSize()[1] * 2

config.view()
f = config.getFrame(CAMERA_NAME)
config.view_setCamera(f)

q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [4]:
def solve_touch_with_komo(komo_config):
    komo = ry.KOMO(komo_config, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, scale=[1e2])
    komo.addObjective([], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
    if ret:
        path = komo.getPath()
        return path
    else:
        return None

In [5]:
ry.params_add({
    "rrt/stepsize": 0.05
})
def solve(cur_config : ry.Config, goal_name : str):
    q_begin = cur_config.getJointState()
    q_ego = cur_config.getFrame(EGO_NAME).getPosition()[:2]
    q_object = cur_config.getFrame(OBJ_NAME).getPosition()[:2]
    q_target = cur_config.getFrame(goal_name).getPosition()[:2]

    ITERATIONS = 16
    delta_theta = 2 * np.pi / ITERATIONS
    theta_offset = np.random.uniform(0, 2 * np.pi)
    touch_positions = []
    for i in range(ITERATIONS):
        transpose_polar1 = np.array([GAP, i * delta_theta + theta_offset])
        transpose_cartesian1 = np.array(polar_to_cartesian(transpose_polar1))
        q_home1 = q_object + transpose_cartesian1 - q_ego + q_begin
        cur_config.setJointState(q_home1)
        # view config
        # cur_config.view_setCamera(cur_config.getFrame(CAMERA_NAME))
        # cur_config.view(True)

        q_sub = solve_touch_with_komo(cur_config)
        if q_sub is not None:
            touch_positions.append(q_sub)

    # reshuffle touch positions
    np.random.shuffle(touch_positions)

    for q_sub in touch_positions:
        cur_config.setJointState(q_begin)
        copy_config = ry.Config()
        copy_config.addConfigurationCopy(cur_config)
        copy_config.setJointState(q_sub)
        if copy_config.getCollisionsTotalPenetration() > 0:
            continue

        rrt = ry.PathFinder()
        rrt.setProblem(cur_config, [q_begin], [q_sub])
        solution1 = rrt.solve()
        del rrt
        if not solution1.feasible:
            continue
        
        cur_config.setJointState(q_sub)
        cur_config.attach(EGO_NAME, OBJ_NAME)
        q_real_goal = q_target + q_sub - q_object - q_ego + q_begin

        rrt = ry.PathFinder()
        rrt.setProblem(cur_config, [q_sub], [q_real_goal])
        solution2 = rrt.solve()
        del rrt
        cur_config.attach(FLOOR_NAME, OBJ_NAME)
        cur_config.setJointState(q_begin)
        if solution1.feasible and solution2.feasible:
            return solution1, solution2
        
    return None, None

In [6]:
def rrt_path(cur_config : ry.Config, q_target : np.ndarray):
    rrt = ry.PathFinder()
    rrt.setProblem(cur_config, [cur_config.getJointState()], [q_target]) # WRONG
    solution = rrt.solve()
    if solution.feasible:
        return solution.x
    else:
        return None

In [7]:
# forward subproblem search algorithm
objects = [OBJ_NAME]
def forward_subproblem_search(start_config : ry.Config):
    start_node = SolutionNode(start_config)
    L = SolutionTree(start_node)
    
    view_config = ry.Config()

    while L.get_len() > 0:
        cur_node = L.get_next_node() # get the next best one
        x = cur_node.get_config()

        view_config.clear()
        view_config.addConfigurationCopy(x)
        view_config.view_setCamera(view_config.getFrame(CAMERA_NAME))
        view_config.view(message=f"""node_depth: {cur_node.depth}
tree_depth: {L.get_depth()}
tree_mode: {L.mode}""")

        sol1, sol2 = solve(x, GOAL_NAME)
        if sol1 is not None:
            del view_config
            return cur_node.get_path()
        
        for object in objects: # there is just one object for now
            object_frame = x.getFrame(object)
            if not reachable(x, object_frame):
                continue
            inner_radius = 0.5
            outer_radius = 3.0
            step_size = 0.05
            bottleneck_threshold = 0.25
            vertex_count = 200
            bottleneck_vertices = bottleneck_vertices_from_config(x, inner_radius, outer_radius, step_size, bottleneck_threshold=bottleneck_threshold)
            subgoals = vertex_to_position(bottleneck_vertices, step_size)
            display_points(x, subgoals) # for testing purposes
            for subgoal in subgoals:
                new_config = ry.Config()
                new_config.addConfigurationCopy(x)
                new_config.getFrame(SUB_GOAL_NAME).setPosition(subgoal.tolist() + [0.2])
                if reject(new_config, L):
                    # new_config.view_setCamera(new_config.getFrame(CAMERA_NAME))
                    # new_config.view(True, message="REJECTED")
                    # new_config.view_close()
                    del new_config
                    continue

                path1, path2 = solve(new_config, SUB_GOAL_NAME)
                if path1 is not None:
                    new_config.setJointState(path1.x[-1])
                    new_config.attach(EGO_NAME, OBJ_NAME)
                    new_config.setJointState(path2.x[-1])
                    new_config.attach(FLOOR_NAME, OBJ_NAME)
                    new_node = cur_node.add_child(new_config)
                    L.add_node(new_node)
                else:
                    del new_config
    del view_config
    return None

In [ ]:
path = forward_subproblem_search(config)

print(len(path))

In [ ]:
save_pngs = False
for i in range(1, len(path)):
    next_config = path[i].get_config()
    config.getFrame(SUB_GOAL_NAME).setPosition(next_config.getFrame(OBJ_NAME).getPosition())
    solution1, solution2 = None, None
    while solution1 is None:
        solution1, solution2 = solve(config, SUB_GOAL_NAME)
            
    animate_solution(config, solution1, save_pngs=save_pngs, png_path=f"temp/{PUZZLE_NAME}-p-{i}-1")
    config.attach(EGO_NAME, OBJ_NAME)
    animate_solution(config, solution2, save_pngs=save_pngs, png_path=f"temp/{PUZZLE_NAME}-p-{i}-2")
    config.attach(FLOOR_NAME, OBJ_NAME)

solution1, solution2 = None, None
while solution1 is None:
    solution1, solution2 = solve(config, GOAL_NAME)
    
animate_solution(config, solution1, save_pngs=save_pngs, png_path=f"temp/{PUZZLE_NAME}-z-1")
config.attach(EGO_NAME, OBJ_NAME)
animate_solution(config, solution2, save_pngs=save_pngs, png_path=f"temp/{PUZZLE_NAME}-z-2")
config.attach(FLOOR_NAME, OBJ_NAME)

In [10]:
config.view_close()
del config